In [106]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode(connected=True)
import pandas as pd
import numpy as np

plotly.offline.init_notebook_mode(connected=True)

In [107]:
df = pd.read_csv('cleaned_data.csv')
# df.shape
df.head(3)

,Unnamed: 0,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,State.1,County,Zip,Country,Congressional District,status,statusText,color
0,0,Aurora,Illinois,41.747452,-88.238622,1.0,place,IL,DuPage County,60504.0,US,IL11,0,Ordinance Active - With No Response,"rgb(255, 0, 0)"
1,1,Carbondale,Illinois,37.739004,-89.209109,1.0,place,IL,Jackson County,62901.0,US,IL12,0,Ordinance Active - With No Response,"rgb(255, 0, 0)"
2,2,Champaign,Illinois,40.107890,-88.244266,1.0,place,IL,Champaign County,61820.0,US,IL13,0,Ordinance Active - With No Response,"rgb(255, 0, 0)"


I'm going to update my values here. I won't persist these changes because a city's status may change multiple times and the size of the data set is very small, so this step takes very little time. 

In [108]:
updates_df = pd.read_csv('updates/updates_9_19_18.csv')
updates_df.head()

,City,State,MAP_STATUS,Unnamed: 3,Unnamed: 4
0,Aurora,Illinois,0,NaN,NaN
1,Carbondale,Illinois,0,NaN,Please put the color's number in the MAP_STATU...
2,Champaign,Illinois,0,NaN,NaN
3,Chicago,Illinois,0,NaN,Red = 0
4,Cicero,Illinois,0,NaN,Pink = 1


In [109]:
df['status'] = updates_df['MAP_STATUS']

df.loc[df['status'] == 4]

,Unnamed: 0,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,State.1,County,Zip,Country,Congressional District,status,statusText,color
12,12,Peoria,Illinois,40.693137,-89.589847,1.0,place,IL,Peoria County,61601.0,US,IL17,4,Ordinance Active - With No Response,"rgb(255, 0, 0)"
63,63,Blue River,Colorado,40.087476,-108.804829,0.0,NaN,CO,NaN,NaN,NaN,CO3,4,Ordinance Active - With No Response,"rgb(255, 0, 0)"


So far so good

In [110]:
# Updating Ordinance Status

d_text = {0: 'Ordinance Active - With No Response', 
          1: 'Ordinance Active - With Response Indicating No Immediate Repeal',
          2: 'Ordinance Active - With Committment To Review', 
          3: 'Ordinance Halted - With Committment to Review',
          4: 'Ordinance Repealed'}

df['statusText'] = df['status'].map(d_text)

# Updating Marker Color 

d_color = {0: 'rgba(255, 0, 0, 1.0)',
           1: 'rgba(255, 192, 203, 1.0)',
           2: 'rgba(255, 165, 0, 1.0)', 
           3: 'rgba(255, 255, 0, 1.0)', 
           4: 'rgba(127, 255, 0, 1.0)'}

df['color'] = df['status'].map(d_color)

# Creating Marker Line Color Primarily for Subplot Workaround 

d_line_color = {0: 'rgba(0, 0, 0, 1.0)', 1: 'rgba(0, 0, 0, 1.0)',
                2: 'rgba(0, 0, 0, 1.0)', 3: 'rgba(0, 0, 0, 1.0)',
                4: 'rgba(0, 0, 0, 1.0)'}

df['linecolor'] = df['status'].map(d_line_color)

Let's check to see if that works

In [111]:
df.loc[df['color'] == 'rgba(127, 255, 0, 1.0)']

,Unnamed: 0,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,State.1,County,Zip,Country,Congressional District,status,statusText,color,linecolor
12,12,Peoria,Illinois,40.693137,-89.589847,1.0,place,IL,Peoria County,61601.0,US,IL17,4,Ordinance Repealed,"rgba(127, 255, 0, 1.0)","rgba(0, 0, 0, 1.0)"
63,63,Blue River,Colorado,40.087476,-108.804829,0.0,NaN,CO,NaN,NaN,NaN,CO3,4,Ordinance Repealed,"rgba(127, 255, 0, 1.0)","rgba(0, 0, 0, 1.0)"


Looks good.

In [112]:
# This creates the text that appears at each point on the the scroll over
df['text'] = df['City'] + ', ' + df['State.1'] + '.' + ' ' + df['statusText']

#### And now for the plotting!

In [120]:
# creating my trace dataframe
new_columns = ['lat', 'lon', 'text', 'status', 'statusText', 'color']

city_plots = pd.DataFrame(data = [df['Latitude'], df['Longitude'], df['text'], df['status'], df['statusText'], df['color']])
final_plots = city_plots.T
final_plots.columns = new_columns

# I created a 'subtitle' by plotting a sixth 'phantom' trace 
# However, it didn't look great, so I opted not to include it for now

subplot_text = ("<br> ^ Legend categories are toggle-able. Feel free to click them on and off. <br><br><b>Note:</b> This is not a complete map of communities with unconstitutional <br> anti-panhandling ordinances. Ordinances were targeted based <br> on local partnerships and resources. <br><br> More than <i>60%</i> of communities surveyed in a 2016 survey <br> had some form of panhandling ordinance. <br>")

final_plots = final_plots.append({'lat': 44.49, 'lon': -66.57, 'text': 'subtitle', 'status': 5, 
                    'statusText': subplot_text, 'color':'rgba(255, 255, 255, 0.0)'}, ignore_index=True)
d_text[5] = subplot_text
d_color[5] = 'rgba(255, 255, 255, 0.0)'
d_line_color[5] = 'rgba(0, 0, 0, 0.0)'

# organizing the data by desired trace
traces_for_plot = []

for i in range(0,6): 
    trace_df = final_plots[final_plots['status'] == i]
    traces_for_plot.append(trace_df)

# building the traces
final_traces = []
i = 0

for trace in traces_for_plot:
    print(i)
    final_trace = go.Scattergeo(
           locationmode = 'USA-states',
            lon = trace['lon'],
            lat = trace['lat'],
            text = trace['text'],
            hoverinfo = 'text',
            mode = 'markers',
            name = d_text[i],
            marker = dict(
                size = 8,
                opacity = 0.85,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color=d_line_color[i]), 
                color = d_color[i]
            ))
    final_traces.append(final_trace)
    i += 1
    
    
layout = go.Layout(showlegend=True, title = 'Targeted Cities with Active Anti-Panhandling Laws', 
                   legend = dict(
#                    orientation='h',
#                    yanchor='top',
#                     xanchor = 'center',
#                     x = 1.5,
                    y = .6,
                   tracegroupgap = 60
                   ),
                   geo = dict(scope = 'usa',
                            projection=dict(type='albers usa'),
                            showland = True,
                            landcolor = "rgb(180, 180, 180)",
                            subunitcolor = "rgb(217, 217, 217)",
                            countrycolor = "rgb(217, 217, 217)",
                            countrywidth = 0.5,
                            subunitwidth = 0.5))
                   
# # Make a figure object
fig = go.Figure(data=final_traces, layout=layout)

# Send to Plotly and show in notebook
# plotly.offline.iplot(fig, validate=False, filename='test1.html')

# Show in Browser and gen
plotly.offline.plot(fig, validate=False, filename='city_locations.html')

0
1
2
3
4
5


'file:///Users/flatironschool/Documents/Projects/NLCHP_Map/Repo/test1.html'

In [72]:
print(len(final_traces))
final_traces[4]

6


Scattergeo({
    'hoverinfo': 'text',
    'lat': array([40.693137 , 40.0874759]),
    'locationmode': 'USA-states',
    'lon': array([ -89.589847 , -108.8048292]),
    'marker': {'color': 'rgba(127, 255, 0, 1.0)',
               'line': {'color': 'rgba(102, 102, 102)', 'width': 1},
               'opacity': 0.85,
               'size': 8,
               'symbol': 'circle'},
    'mode': 'markers',
    'name': 'Ordinance Repealed',
    'text': array(['Peoria, IL. Ordinance Repealed', 'Blue River, CO. Ordinance Repealed'],
                  dtype='<U34')
})